<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a7c2070389535cbdbf7a6dc977a03b0a2bfcf608b53094ab7357a0c98d677fc4
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-20 09:13:16
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.41 C
-------------------
Today PnL: 43.34 K (0.31%)
Current PnL: -20.80 L (-13.86%)
CY Booked + Current PnL: -8.90 L (-5.93%)
-------------------
Total profit:  1.94 L
Total loss:  -22.74 L
-------------------
Total Booked + Current PnL: 18.08 L (14.68%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.46%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.97 L (62.53%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 7.88%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-18.41,63.0,X-LC,3.96,217830.0,13330.0,9955.0,0.79,6.52,4.57,11.39,10.0,1.34,1.56,16.05,X40N,ATH,PHARMA
56,RELIANCE,1533.00,-10.29,69.0,X-LC,2.85,224640.0,14274.0,14512.0,1.64,6.79,6.46,13.68,37.0,0.98,1.61,24.43,XY25,NTT,REFINERIES
76,TTKPRESTIG,770.00,96.37,49.0,M-SC,1.68,84567.0,-16210.0,16305.0,0.52,-16.09,19.28,0.09,245.0,-0.99,0.61,11.10,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.58,58.0,X-MC,1.49,213000.0,21258.0,16508.0,-0.46,11.09,7.75,19.69,99.0,1.29,1.53,18.64,XY25,NTT,AC
33,ICICIGI,2260.25,-13.38,65.0,X-MC,2.66,148888.0,12755.0,18373.0,0.55,9.37,12.34,22.86,91.0,0.69,1.07,23.16,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SFL,1287.00,32.67,49.0,M-SC,7.54,175182.0,-87036.0,156858.0,1.73,-33.19,89.54,26.63,229.0,-0.55,1.25,8.92,XY24,NTT,MISC
56,RELIANCE,1533.00,-10.29,69.0,X-LC,2.85,224640.0,14274.0,14512.0,1.64,6.79,6.46,13.68,37.0,0.98,1.61,24.43,XY25,NTT,REFINERIES
54,RECLTD,446.00,47.87,55.0,M-LC,6.02,208725.0,5555.0,36569.0,1.21,2.73,17.52,20.74,55.0,0.15,1.50,8.43,XY25,NTT,FINANCE
48,MASFIN,398.61,-20.51,44.0,H-SC,9.42,90300.0,-7680.0,29284.0,1.21,-7.84,32.43,22.05,152.0,-0.26,0.65,31.26,XR,ATH,FINANCE
11,BANKINDIA,190.00,-25.47,56.0,H-MC,12.34,191496.0,11688.0,100727.0,1.14,6.50,52.60,62.52,88.0,0.12,1.37,42.80,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-56.13,36.0,H-SC,7.72,122430.0,-906.0,132530.0,-1.45,-0.73,108.25,106.72,119.0,-0.01,0.88,22.47,AR,ATH,MISC
38,INDUSINDBK,1800.00,245.24,48.0,L-MC,10.81,42351.0,-34856.0,60249.0,-1.07,-45.15,142.26,32.89,258.0,-0.58,0.30,16.63,XR,NTT,BANKS
6,ASIANTILES,137.00,6722.22,55.0,L-SC,6.37,74827.0,-18983.0,95599.0,-1.05,-20.24,127.76,81.67,269.0,-0.20,0.54,44.90,XR,NTT,CERAMICS
84,WHIRLPOOL,2270.00,-39.15,66.0,M-SC,4.12,102742.0,11244.0,67512.0,-1.05,12.29,65.71,86.07,223.0,0.17,0.74,49.20,XR,NTT,DURABLES
28,HAVELLS,2069.16,-6.89,41.0,X-MC,2.39,230880.0,-16851.0,91913.0,-0.77,-6.80,39.81,30.30,92.0,-0.18,1.65,5.59,X40,ATH,ELECTRICAL


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-187.50,70.0,M-SC,19.60,150300.0,1080.0,143101.0,0.14,0.72,95.21,96.62,208.0,0.01,1.08,62.66,XY25,NTT,FINANCE
59,SAIL,228.00,41.14,46.0,M-MC,10.83,223171.0,-1791.0,169454.0,0.71,-0.80,75.93,74.53,192.0,-0.01,1.60,31.20,XY24,BTT,STEEL
36,INDIAMART,4810.62,-56.13,36.0,H-SC,7.72,122430.0,-906.0,132530.0,-1.45,-0.73,108.25,106.72,119.0,-0.01,0.88,22.47,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,107.70,60.0,H-SC,12.67,131544.0,-10935.0,30452.0,0.00,-7.67,23.15,13.70,163.0,-0.36,0.94,55.26,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.69,63.0,H-SC,1.97,229500.0,-40167.0,76492.0,0.69,-14.90,33.33,13.47,138.0,-0.53,1.64,16.69,XY24,NTT,PAINTS
76,TTKPRESTIG,770.0,96.37,49.0,M-SC,1.68,84567.0,-16210.0,16305.0,0.52,-16.09,19.28,0.09,245.0,-0.99,0.61,11.10,OX40N,NTT,DURABLES
17,CERA,9475.0,-23.39,44.0,H-SC,2.46,139495.0,-36408.0,78424.0,1.05,-20.70,56.22,23.89,149.0,-0.46,1.00,20.47,OX40N,NTT,CERAMICS
65,SIS,528.0,2034.72,47.0,H-SC,3.84,86152.0,-24880.0,48486.0,0.52,-22.41,56.28,21.26,156.0,-0.51,0.62,16.40,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,55.0,M-LC,6.02,208725.0,5555.0,36569.0,1.21,2.73,17.52,20.74,55.0,0.15,1.50,8.43,XY25,NTT,FINANCE
31,HINDZINC,730.22,33.63,61.0,M-LC,10.26,217500.0,12424.0,100137.0,-0.05,6.06,46.04,54.89,52.0,0.12,1.56,31.61,X5K,ATH,METALS
78,UNIONBANK,163.00,-13.18,52.0,M-LC,8.63,156554.0,15714.0,28775.0,0.73,11.16,18.38,31.59,66.0,0.55,1.12,40.60,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,56.0,H-MC,12.34,191496.0,11688.0,100727.0,1.14,6.50,52.60,62.52,88.0,0.12,1.37,42.80,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-39.15,66.0,M-SC,4.12,102742.0,11244.0,67512.0,-1.05,12.29,65.71,86.07,223.0,0.17,0.74,49.20,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-39.15,66.0,M-SC,4.12,102742.0,11244.0,67512.0,-1.05,12.29,65.71,86.07,223.0,0.17,0.74,49.20,XR,NTT,DURABLES
11,BANKINDIA,190.00,-25.47,56.0,H-MC,12.34,191496.0,11688.0,100727.0,1.14,6.50,52.60,62.52,88.0,0.12,1.37,42.80,XR,NTT,BANKS
31,HINDZINC,730.22,33.63,61.0,M-LC,10.26,217500.0,12424.0,100137.0,-0.05,6.06,46.04,54.89,52.0,0.12,1.56,31.61,X5K,ATH,METALS
85,WIPRO,420.00,-15.15,43.0,M-LC,6.00,149358.0,-1587.0,111048.0,0.00,-1.05,74.35,72.51,53.0,-0.01,1.07,4.91,XR,NTT,IT
36,INDIAMART,4810.62,-56.13,36.0,H-SC,7.72,122430.0,-906.0,132530.0,-1.45,-0.73,108.25,106.72,119.0,-0.01,0.88,22.47,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.83,15.0,X-LC,9.58,145218.0,-131305.0,241381.0,0.87,-47.48,166.22,39.81,54.0,-0.54,1.04,2.35,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,32.0,X-SC,38.80,51268.0,-13738.0,164668.0,0.00,-21.13,321.19,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
28,HAVELLS,2069.16,-6.89,41.0,X-MC,2.39,230880.0,-16851.0,91913.0,-0.77,-6.80,39.81,30.30,92.0,-0.18,1.65,5.59,X40,ATH,ELECTRICAL
9,BAJAJHFL,181.50,-14.90,43.0,X-MC,5.45,179514.0,-20992.0,118138.0,0.03,-10.47,65.81,48.45,90.0,-0.18,1.29,3.52,X40N,ATH,FINANCE
55,RELAXO,1176.00,-39.49,43.0,X-SC,4.53,82513.0,-62647.0,138572.0,0.15,-43.16,167.94,52.31,136.0,-0.45,0.59,10.45,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.83,45.0,H-LC,0.37,156245.0,-29850.0,77326.0,0.12,-16.04,49.49,25.51,15.0,-0.39,1.12,15.14,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,1.58,58.0,X-MC,1.49,213000.0,21258.0,16508.0,-0.46,11.09,7.75,19.69,99.0,1.29,1.53,18.64,XY25,NTT,AC
28,HAVELLS,2069.16,-6.89,41.0,X-MC,2.39,230880.0,-16851.0,91913.0,-0.77,-6.80,39.81,30.30,92.0,-0.18,1.65,5.59,X40,ATH,ELECTRICAL
41,ITC,452.00,-37.50,60.0,X-LC,2.61,201737.0,1599.0,19286.0,0.10,0.80,9.56,10.44,4.0,0.08,1.45,6.41,X40,NTT,FMCG
33,ICICIGI,2260.25,-13.38,65.0,X-MC,2.66,148888.0,12755.0,18373.0,0.55,9.37,12.34,22.86,91.0,0.69,1.07,23.16,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,32.0,X-SC,38.80,51268.0,-13738.0,164668.0,0.00,-21.13,321.19,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-53.83,15.0,X-LC,9.58,145218.0,-131305.0,241381.0,0.87,-47.48,166.22,39.81,54.0,-0.54,1.04,2.35,XY24,NTT,AUTO
2,ACC,3906.00,-38.07,48.0,X-MC,3.21,184440.0,-53511.0,206167.0,0.64,-22.49,111.78,64.15,174.0,-0.26,1.32,3.46,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-14.90,43.0,X-MC,5.45,179514.0,-20992.0,118138.0,0.03,-10.47,65.81,48.45,90.0,-0.18,1.29,3.52,X40N,ATH,FINANCE
73,TCS,4389.97,-29.65,48.0,X-LC,12.77,280637.0,-65319.0,132012.0,0.79,-18.88,47.04,19.28,1.0,-0.49,2.01,3.75,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,32.0,X-SC,38.80,51268.0,-13738.0,164668.0,0.00,-21.13,321.19,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-39.49,43.0,X-SC,4.53,82513.0,-62647.0,138572.0,0.15,-43.16,167.94,52.31,136.0,-0.45,0.59,10.45,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.87,44.0,X-SC,4.29,95865.0,-32805.0,78101.0,0.41,-25.50,81.47,35.20,219.0,-0.42,0.69,10.03,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.83,48.0,X-SC,0.82,107100.0,5096.0,38834.0,0.77,5.00,36.26,43.06,122.0,0.13,0.77,22.97,X40N,ATH,MISC
32,HONAUT,58357.33,-22.75,48.0,X-SC,2.56,108930.0,-19008.0,66142.0,0.00,-14.86,60.72,36.84,143.0,-0.29,0.78,11.52,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.65,48.0,X-LC,12.77,280637.0,-65319.0,132012.0,0.79,-18.88,47.04,19.28,1.0,-0.49,2.01,3.75,X40,ATH,IT
39,INFY,2275.00,-20.67,43.0,X-LC,7.65,309638.0,-3250.0,174945.0,0.87,-1.04,56.50,54.87,3.0,-0.02,2.22,5.54,X40,BTT,IT
41,ITC,452.00,-37.50,60.0,X-LC,2.61,201737.0,1599.0,19286.0,0.10,0.80,9.56,10.44,4.0,0.08,1.45,6.41,X40,NTT,FMCG
82,VBL,671.64,-17.78,55.0,X-LC,10.45,295904.0,-19938.0,132595.0,0.49,-6.31,44.81,35.67,5.0,-0.15,2.12,6.67,X40N,ATH,FMCG
1,ABB,7934.00,-39.96,52.0,H-LC,6.89,250944.0,-10675.0,129889.0,0.50,-4.08,51.76,45.57,7.0,-0.08,1.80,6.78,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6722.22,55.0,L-SC,6.37,74827.0,-18983.0,95599.0,-1.05,-20.24,127.76,81.67,269.0,-0.20,0.54,44.90,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,29.62,81350.0,-32199.0,72247.0,-0.00,-28.36,88.81,35.27,268.0,-0.45,0.58,99.11,XR,NTT,HOTELS
48,MASFIN,398.61,-20.51,44.0,H-SC,9.42,90300.0,-7680.0,29284.0,1.21,-7.84,32.43,22.05,152.0,-0.26,0.65,31.26,XR,ATH,FINANCE
84,WHIRLPOOL,2270.00,-39.15,66.0,M-SC,4.12,102742.0,11244.0,67512.0,-1.05,12.29,65.71,86.07,223.0,0.17,0.74,49.20,XR,NTT,DURABLES
45,KPIGREEN,731.05,2.11,60.0,H-SC,5.94,121577.0,-3720.0,62649.0,0.84,-2.97,51.53,47.03,141.0,-0.06,0.87,52.23,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-187.50,70.0,M-SC,19.60,150300.0,1080.0,143101.0,0.14,0.72,95.21,96.62,208.0,0.01,1.08,62.66,XY25,NTT,FINANCE
67,STARHEALTH,761.00,23.22,73.0,H-SC,9.89,274720.0,-6403.0,139256.0,0.21,-2.28,50.69,47.26,144.0,-0.05,1.97,47.19,XY24,NTT,INSURANCE
4,ANGELONE,3033.00,16.33,70.0,X-SC,8.28,205722.0,14716.0,42975.0,0.49,7.70,20.89,30.21,157.0,0.34,1.47,30.19,X40N,NTT,FINANCE
31,HINDZINC,730.22,33.63,61.0,M-LC,10.26,217500.0,12424.0,100137.0,-0.05,6.06,46.04,54.89,52.0,0.12,1.56,31.61,X5K,ATH,METALS
63,SHALBY,327.00,1148.80,50.0,M-SC,23.38,169785.0,-11358.0,56827.0,-0.51,-6.27,33.47,25.10,235.0,-0.20,1.22,36.07,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.65
1,25,44.24
2,50,75.90


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.34
LC    33.89
MC    21.79
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.59
X40      14.80
XY25     12.42
XR       10.88
X40N     10.26
AR        8.07
OX40N     7.64
X200      1.81
X5K       1.56
SR        1.12
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.40
X-LC    22.88
X-MC    16.11
M-SC    12.81
M-LC     5.25
H-LC     4.73
X-SC     4.67
H-MC     3.78
M-MC     1.60
L-SC     1.46
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.29,-2.95,36.89
IT,12.73,-18.35,82.23
FINANCE,12.17,-11.42,61.92
MISC,6.89,-19.58,85.84
BANKS,6.33,-12.73,72.04
PAINTS,5.73,-15.32,32.51
ELECTRICAL,5.35,-10.23,48.90
INSURANCE,4.01,1.02,35.98
AUTO,2.82,-46.13,115.99


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3303835.0,22
XR,1388930.0,14
AR,1204675.0,9
X40,772388.0,10
X40N,594907.0,8
OX40N,581914.0,10
XY25,449818.0,8
SR,264770.0,2
X5K,100137.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3453933.0,25
M-SC,1254911.0,15
X-MC,1079654.0,11
X-LC,1043218.0,13
X-SC,529292.0,6
H-MC,351421.0,3
H-LC,279966.0,3
M-LC,276529.0,4
L-SC,258205.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1186133.0      6
           AR         835223.0      5
           XR         831903.0      7
M-SC       XY24       686152.0      6
X-MC       XY24       581711.0      4
X-LC       X40        478380.0      5
           XY24       308284.0      2
X-SC       X40N       286523.0      4
H-SC       OX40N      273255.0      4
           SR         264770.0      2
X-MC       X40        215907.0      4
H-LC       AR         207215.0      2
X-LC       X40N       190246.0      3
H-MC       XY24       178658.0      1
M-MC       XY24       169454.0      1
L-SC       XR         167846.0      2
X-SC       XY24       164668.0      1
X-MC       XY25       163898.0      2
M-SC       AR         162237.0      2
           OX40N      146264.0      4
           XY25       143101.0      1
X-MC       X40N       118138.0      1
M-SC       XR         117157.0      2
M-LC       XR         111048.0      1
H-MC       XR         100727.0      1
M-LC       X5K        100137.0      1
L-SC       OX40N       90359.0      1
X-SC       X40         78101.0      1
H-LC       X200        72751.0      1
H-MC       OX40N       72036.0      1
X-LC       XY25        66308.0      3
H-SC       MH          62649.0      1
L-MC       XR          60249.0      1
L-LC       XY25        39942.0      1
M-LC       XY25        36569.0      1
           XY24        28775.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
